# DDM 
Double-diode model with the RTFrance dataset.

In [11]:
using PSO, CUDA, Random

const data_dir = "../data/"
include("utils.jl")

benchmark (generic function with 1 method)

In [12]:
# set random seed: for reproduction
Random.seed!(CURAND.default_rng(), 123)

## Read data and set up search range

In [13]:
# select data type to be used on GPU: Float64 or Float32
# Float32 tends to be a little faster, but with lower precision
const TF = Float64  

Float64

In [14]:
const T = TF(33 + 273.15)  # temperature in Kalvin of the RTC France case

const data = read_data(joinpath(data_dir, "RTFrance.csv"), TF);

In [15]:
const x_min = TF[0, 0, 0, 1, 1, 0,   0]
const x_max = TF[1, 1, 1, 2, 2, 0.5, 100];

## PSO parameters


In [16]:
const w = TF(0.5)
const c1 = TF(2.5)
const c2 = TF(1.6)
const np = 6400
const niter = 400
const min_delta=TF(1e-10)
const patience=30
const threads=640   # threads per block in CUDA
;

## Run PSO once as an example
- Known best minimum in the literature: 2.50972e-05
- Note that the SSE (sum of square error) is minimized, which is equivalent to the minimization of RMSE.

In [17]:
@time pso(data, T, :DDM, x_min, x_max; niter, np, w, c1, c2, log=true, threads, min_delta, patience)

(threads, blocks) = (640, 10)
iter      sse       
1         7.46857e-02
2         1.44321e-02
3         1.30618e-02
4         8.64379e-03
5         3.29697e-03
6         1.31562e-03
7         3.46812e-04
8         3.46812e-04
9         3.31199e-04
10        3.15236e-04
11        3.09370e-04
12        2.74197e-04
13        2.44367e-04
14        2.31656e-04
15        1.96355e-04
16        1.74083e-04
17        1.43863e-04
18        1.18640e-04
19        1.13296e-04
20        8.63508e-05
21        6.39225e-05
22        5.51917e-05
23        4.59121e-05
24        3.55611e-05
25        3.32233e-05
26        3.31928e-05
27        3.10449e-05
28        2.90724e-05
29        2.88931e-05
30        2.69346e-05
31        2.69346e-05
32        2.68975e-05
33        2.63511e-05
34        2.57078e-05
35        2.55308e-05
36        2.54557e-05
37        2.54089e-05
38        2.54089e-05
39        2.54089e-05
40        2.54089e-05
41        2.54003e-05
42        2.53752e-05
43        2.53710e-05
44 

(sse = 2.509718858385086e-5, solution = [0.7607810800699066, 0.7493500426117496, 0.22597396285949586, 1.9999999999473286, 1.4510166534515405, 0.03674043177002697, 55.48542097049508], stopped_iter = 133)

## Measure statistics
Run PSO multiple times and record the results. Analyze the results.

In [18]:
f = () -> pso(data, T, :DDM, x_min, x_max; niter, np, w, c1, c2, log=false, threads, min_delta, patience)
measure_RMSE_stats(f, size(data, 2); n_trials=30)

Dict{Symbol, Float64} with 4 entries:
  :max  => 0.000986022
  :std  => 1.34365e-6
  :mean => 0.000983227
  :min  => 0.000982485

## Benchmark the running time
Run the PSO multiple times and measure the average running time. [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) is a library devoted to benchmarking in the `benchmark` function defined in `utils.jl`.

Due to GPU characteristics as well as early stopping, the variance among run time can be fairly high.

In [20]:
benchmark(f; n_trials=100)

BenchmarkTools.Trial: 
  memory estimate:  956.65 KiB
  allocs estimate:  21247
  --------------
  minimum time:     23.713 ms (0.00% GC)
  median time:      219.493 ms (0.00% GC)
  mean time:        258.518 ms (0.72% GC)
  maximum time:     669.978 ms (1.53% GC)
  --------------
  samples:          100
  evals/sample:     1